In [1]:
!pip install google-api-python-client pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 4.5 MB/s eta 0:00:00a 0:00:01


In [2]:
import math

# Define a function to calculate total quota usage for a given number of videos
def calculate_quota_usage(num_videos):
    # Quota for video search
    video_search_requests = math.ceil(num_videos / 50)
    video_search_quota = video_search_requests * 500
    
    # Quota for comments (1 unit per video to get top 100 comments)
    comments_quota = num_videos * 1
    
    # Total quota usage
    total_quota = video_search_quota + comments_quota
    
    return total_quota

# Now, find the maximum number of videos that keep total quota under 9000 units
target_quota = 9000
max_videos = 0

for videos in range(1, 5000):  # Try different values for videos
    total_quota = calculate_quota_usage(videos)
    if total_quota <= target_quota:
        max_videos = videos
    else:
        break

max_videos, calculate_quota_usage(max_videos)  # Return the number of videos and the total quota usage

(800, 8800)

In [5]:
import pandas as pd
import os
import time
import logging
from googleapiclient.discovery import build
import config  # Import the config file
from googleapiclient.errors import HttpError

# Set up your YouTube API key
api_key = config.YOUTUBE_API_KEY

# Build the YouTube API client
youtube = build('youtube', 'v3', developerKey=api_key)

# Set up logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Ensure a directory exists to store the files
output_folder = 'youtube_data'
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

# Function to get video details by a search term (with pagination and error handling)
def get_video_details(search_query, max_results=800):
    videos = []
    next_page_token = None

    logging.info(f"Starting to scrape video details for search term: {search_query}")

    while len(videos) < max_results:
        try:
            request = youtube.search().list(
                q=search_query,
                part='snippet',
                type='video',
                maxResults=min(50, max_results - len(videos)),  # Fetch up to 50 results per request
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response['items']:
                video = {
                    'video_id': item['id']['videoId'],
                    'title': item['snippet']['title'],
                    'published_at': item['snippet']['publishedAt'],
                    'channel_title': item['snippet']['channelTitle']
                }
                videos.append(video)

            logging.info(f"Scraped {len(videos)} videos so far...")

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break  # No more pages, stop the loop

        except Exception as e:
            logging.error(f"Error occurred while fetching videos: {e}")
            time.sleep(5)  # Sleep for 5 seconds and retry

    logging.info(f"Finished scraping video details. Total videos scraped: {len(videos)}")
    return pd.DataFrame(videos)

# Function to get top 500 comments from a video (with pagination and error handling)
def get_video_comments(video_id, max_results=500):
    comments = []
    next_page_token = None

    logging.info(f"Starting to scrape comments for video: {video_id}")

    while len(comments) < max_results:
        try:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=min(100, max_results - len(comments)),  # Fetch up to 100 comments per request
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response['items']:
                comment = {
                    'author': item['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                    'text': item['snippet']['topLevelComment']['snippet']['textDisplay'],
                    'published_at': item['snippet']['topLevelComment']['snippet']['publishedAt'],
                    'like_count': item['snippet']['topLevelComment']['snippet']['likeCount']
                }
                comments.append(comment)

            logging.info(f"Scraped {len(comments)} comments so far for video {video_id}...")

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break  # No more pages, stop the loop

        except HttpError as e:
            # Check if the error is due to comments being disabled
            if e.resp.status == 403 and 'commentsDisabled' in str(e):
                logging.warning(f"Encountered 403 Forbidden with reason 'commentsDisabled' for video {video_id}")
                break  # Do not retry for this video
            else:
                logging.error(f"Error occurred while fetching comments for video {video_id}: {e}")
                time.sleep(5)  # Sleep for 5 seconds and retry

    logging.info(f"Finished scraping comments for video {video_id}. Total comments: {len(comments)}")
    return pd.DataFrame(comments)

# Example: Scrape video details for a search term (up to 800 videos)
search_term = 'U.S. elections 2024'
try:
    videos_df = get_video_details(search_term, max_results=800)
    video_file_path = os.path.join(output_folder, 'youtube_videos.csv')
    videos_df.to_csv(video_file_path, index=False)
    logging.info(f"Video details saved to: {video_file_path}")
except Exception as e:
    logging.error(f"Error occurred while saving video details: {e}")

# Scrape top 500 comments for **each video** in the list of scraped videos
for idx, row in videos_df.iterrows():
    try:
        video_id = row['video_id']
        comments_df = get_video_comments(video_id, max_results=500)
        comments_file_path = os.path.join(output_folder, f'youtube_comments_{video_id}.csv')
        comments_df.to_csv(comments_file_path, index=False)
        logging.info(f"Comments for video {video_id} saved to: {comments_file_path}")
    except Exception as e:
        logging.error(f"Error occurred while saving comments for video {video_id}: {e}")

2024-10-15 11:36:28,048 - INFO - file_cache is only supported with oauth2client<4.0.0
2024-10-15 11:36:28,054 - INFO - Starting to scrape video details for search term: U.S. elections 2024
2024-10-15 11:36:28,854 - INFO - Scraped 50 videos so far...
2024-10-15 11:36:29,543 - INFO - Scraped 100 videos so far...
2024-10-15 11:36:30,173 - INFO - Scraped 150 videos so far...
2024-10-15 11:36:30,985 - INFO - Scraped 200 videos so far...
2024-10-15 11:36:31,699 - INFO - Scraped 250 videos so far...
2024-10-15 11:36:32,209 - INFO - Scraped 300 videos so far...
2024-10-15 11:36:32,928 - INFO - Scraped 350 videos so far...
2024-10-15 11:36:33,607 - INFO - Scraped 400 videos so far...
2024-10-15 11:36:34,360 - INFO - Scraped 450 videos so far...
2024-10-15 11:36:35,184 - INFO - Scraped 500 videos so far...
2024-10-15 11:36:35,998 - INFO - Scraped 550 videos so far...
2024-10-15 11:36:36,820 - INFO - Scraped 589 videos so far...
2024-10-15 11:36:36,822 - INFO - Finished scraping video details. To

2024-10-15 11:36:42,352 - INFO - Finished scraping comments for video l66otqSCgGg. Total comments: 27
2024-10-15 11:36:42,357 - INFO - Comments for video l66otqSCgGg saved to: youtube_data/youtube_comments_l66otqSCgGg.csv
2024-10-15 11:36:42,358 - INFO - Starting to scrape comments for video: I6cSAbabGac
2024-10-15 11:36:42,512 - INFO - Scraped 9 comments so far for video I6cSAbabGac...
2024-10-15 11:36:42,513 - INFO - Finished scraping comments for video I6cSAbabGac. Total comments: 9
2024-10-15 11:36:42,516 - INFO - Comments for video I6cSAbabGac saved to: youtube_data/youtube_comments_I6cSAbabGac.csv
2024-10-15 11:36:42,517 - INFO - Starting to scrape comments for video: z1XLk5Ba0qc
2024-10-15 11:36:42,784 - INFO - Scraped 100 comments so far for video z1XLk5Ba0qc...
2024-10-15 11:36:43,016 - INFO - Scraped 200 comments so far for video z1XLk5Ba0qc...
2024-10-15 11:36:43,244 - INFO - Scraped 300 comments so far for video z1XLk5Ba0qc...
2024-10-15 11:36:43,491 - INFO - Scraped 400 co

2024-10-15 11:36:50,308 - INFO - Scraped 200 comments so far for video PEXf3b3WPH0...
2024-10-15 11:36:50,499 - INFO - Scraped 232 comments so far for video PEXf3b3WPH0...
2024-10-15 11:36:50,500 - INFO - Finished scraping comments for video PEXf3b3WPH0. Total comments: 232
2024-10-15 11:36:50,510 - INFO - Comments for video PEXf3b3WPH0 saved to: youtube_data/youtube_comments_PEXf3b3WPH0.csv
2024-10-15 11:36:50,511 - INFO - Starting to scrape comments for video: PhrWJZO5LaE
2024-10-15 11:36:50,761 - INFO - Scraped 100 comments so far for video PhrWJZO5LaE...
2024-10-15 11:36:51,044 - INFO - Scraped 200 comments so far for video PhrWJZO5LaE...
2024-10-15 11:36:51,272 - INFO - Scraped 300 comments so far for video PhrWJZO5LaE...
2024-10-15 11:36:51,539 - INFO - Scraped 400 comments so far for video PhrWJZO5LaE...
2024-10-15 11:36:51,762 - INFO - Scraped 500 comments so far for video PhrWJZO5LaE...
2024-10-15 11:36:51,763 - INFO - Finished scraping comments for video PhrWJZO5LaE. Total co

2024-10-15 11:37:00,680 - INFO - Starting to scrape comments for video: 5Z6djzUiI6Y
2024-10-15 11:37:00,950 - INFO - Scraped 100 comments so far for video 5Z6djzUiI6Y...
2024-10-15 11:37:01,203 - INFO - Scraped 200 comments so far for video 5Z6djzUiI6Y...
2024-10-15 11:37:01,436 - INFO - Scraped 300 comments so far for video 5Z6djzUiI6Y...
2024-10-15 11:37:01,671 - INFO - Scraped 400 comments so far for video 5Z6djzUiI6Y...
2024-10-15 11:37:01,907 - INFO - Scraped 500 comments so far for video 5Z6djzUiI6Y...
2024-10-15 11:37:01,909 - INFO - Finished scraping comments for video 5Z6djzUiI6Y. Total comments: 500
2024-10-15 11:37:01,919 - INFO - Comments for video 5Z6djzUiI6Y saved to: youtube_data/youtube_comments_5Z6djzUiI6Y.csv
2024-10-15 11:37:01,920 - INFO - Starting to scrape comments for video: qT3F9bkUJaA
2024-10-15 11:37:02,112 - INFO - Scraped 27 comments so far for video qT3F9bkUJaA...
2024-10-15 11:37:02,113 - INFO - Finished scraping comments for video qT3F9bkUJaA. Total comme

2024-10-15 11:37:09,893 - INFO - Scraped 27 comments so far for video qT3F9bkUJaA...
2024-10-15 11:37:09,894 - INFO - Finished scraping comments for video qT3F9bkUJaA. Total comments: 27
2024-10-15 11:37:09,899 - INFO - Comments for video qT3F9bkUJaA saved to: youtube_data/youtube_comments_qT3F9bkUJaA.csv
2024-10-15 11:37:09,900 - INFO - Starting to scrape comments for video: HrUjS0XArAU
2024-10-15 11:37:10,142 - INFO - Scraped 100 comments so far for video HrUjS0XArAU...
2024-10-15 11:37:10,391 - INFO - Scraped 200 comments so far for video HrUjS0XArAU...
2024-10-15 11:37:10,640 - INFO - Scraped 300 comments so far for video HrUjS0XArAU...
2024-10-15 11:37:10,878 - INFO - Scraped 400 comments so far for video HrUjS0XArAU...
2024-10-15 11:37:11,081 - INFO - Scraped 416 comments so far for video HrUjS0XArAU...
2024-10-15 11:37:11,082 - INFO - Finished scraping comments for video HrUjS0XArAU. Total comments: 416
2024-10-15 11:37:11,089 - INFO - Comments for video HrUjS0XArAU saved to: yo

...

2024-10-15 11:42:12,069 - INFO - Scraped 38 comments so far for video vD5a8zjabHM...
2024-10-15 11:42:12,069 - INFO - Finished scraping comments for video vD5a8zjabHM. Total comments: 38
2024-10-15 11:42:12,072 - INFO - Comments for video vD5a8zjabHM saved to: youtube_data/youtube_comments_vD5a8zjabHM.csv
2024-10-15 11:42:12,073 - INFO - Starting to scrape comments for video: jKzDfXdDaVY
2024-10-15 11:42:12,200 - WARNING - Encountered 403 Forbidden with reason "commentsDisabled"
2024-10-15 11:42:12,201 - WARNING - Encountered 403 Forbidden with reason 'commentsDisabled' for video jKzDfXdDaVY
2024-10-15 11:42:12,201 - INFO - Finished scraping comments for video jKzDfXdDaVY. Total comments: 0
2024-10-15 11:42:12,205 - INFO - Comments for video jKzDfXdDaVY saved to: youtube_data/youtube_comments_jKzDfXdDaVY.csv
2024-10-15 11:42:12,205 - INFO - Starting to scrape comments for video: glIycIiEWtQ
2024-10-15 11:42:12,377 - INFO - Scraped 3 comments so far for video glIycIiEWtQ...
2024-10-15 1

2024-10-15 11:42:20,773 - INFO - Scraped 300 comments so far for video G9uEHmHZyLo...
2024-10-15 11:42:21,009 - INFO - Scraped 400 comments so far for video G9uEHmHZyLo...
2024-10-15 11:42:21,228 - INFO - Scraped 500 comments so far for video G9uEHmHZyLo...
2024-10-15 11:42:21,229 - INFO - Finished scraping comments for video G9uEHmHZyLo. Total comments: 500
2024-10-15 11:42:21,242 - INFO - Comments for video G9uEHmHZyLo saved to: youtube_data/youtube_comments_G9uEHmHZyLo.csv
2024-10-15 11:42:21,243 - INFO - Starting to scrape comments for video: vTqxrruWcbE
2024-10-15 11:42:21,416 - INFO - Scraped 10 comments so far for video vTqxrruWcbE...
2024-10-15 11:42:21,417 - INFO - Finished scraping comments for video vTqxrruWcbE. Total comments: 10
2024-10-15 11:42:21,422 - INFO - Comments for video vTqxrruWcbE saved to: youtube_data/youtube_comments_vTqxrruWcbE.csv
2024-10-15 11:42:21,423 - INFO - Starting to scrape comments for video: YX0P3uGkluw
2024-10-15 11:42:21,622 - INFO - Scraped 26 

2024-10-15 11:42:30,361 - INFO - Starting to scrape comments for video: Vb60TL2xJQc
2024-10-15 11:42:30,633 - INFO - Scraped 100 comments so far for video Vb60TL2xJQc...
2024-10-15 11:42:30,946 - INFO - Scraped 200 comments so far for video Vb60TL2xJQc...
2024-10-15 11:42:31,251 - INFO - Scraped 300 comments so far for video Vb60TL2xJQc...
2024-10-15 11:42:31,502 - INFO - Scraped 400 comments so far for video Vb60TL2xJQc...
2024-10-15 11:42:31,781 - INFO - Scraped 500 comments so far for video Vb60TL2xJQc...
2024-10-15 11:42:31,782 - INFO - Finished scraping comments for video Vb60TL2xJQc. Total comments: 500
2024-10-15 11:42:31,792 - INFO - Comments for video Vb60TL2xJQc saved to: youtube_data/youtube_comments_Vb60TL2xJQc.csv
2024-10-15 11:42:31,794 - INFO - Starting to scrape comments for video: YR9fXdvC3fI
2024-10-15 11:42:32,058 - INFO - Scraped 100 comments so far for video YR9fXdvC3fI...
2024-10-15 11:42:32,311 - INFO - Scraped 200 comments so far for video YR9fXdvC3fI...
2024-10

2024-10-15 11:42:40,451 - INFO - Scraped 500 comments so far for video RDZWfHgPmh8...
2024-10-15 11:42:40,452 - INFO - Finished scraping comments for video RDZWfHgPmh8. Total comments: 500
2024-10-15 11:42:40,465 - INFO - Comments for video RDZWfHgPmh8 saved to: youtube_data/youtube_comments_RDZWfHgPmh8.csv
2024-10-15 11:42:40,466 - INFO - Starting to scrape comments for video: MR55NfV-Dyw
2024-10-15 11:42:40,827 - INFO - Scraped 100 comments so far for video MR55NfV-Dyw...
2024-10-15 11:42:41,155 - INFO - Scraped 200 comments so far for video MR55NfV-Dyw...
2024-10-15 11:42:41,478 - INFO - Scraped 300 comments so far for video MR55NfV-Dyw...
2024-10-15 11:42:41,833 - INFO - Scraped 400 comments so far for video MR55NfV-Dyw...
2024-10-15 11:42:42,112 - INFO - Scraped 500 comments so far for video MR55NfV-Dyw...
2024-10-15 11:42:42,114 - INFO - Finished scraping comments for video MR55NfV-Dyw. Total comments: 500
2024-10-15 11:42:42,123 - INFO - Comments for video MR55NfV-Dyw saved to: 

2024-10-15 11:42:47,132 - INFO - Scraped 300 comments so far for video caH6q0FA564...
2024-10-15 11:42:47,378 - INFO - Scraped 400 comments so far for video caH6q0FA564...
2024-10-15 11:42:47,574 - INFO - Scraped 439 comments so far for video caH6q0FA564...
2024-10-15 11:42:47,575 - INFO - Finished scraping comments for video caH6q0FA564. Total comments: 439
2024-10-15 11:42:47,582 - INFO - Comments for video caH6q0FA564 saved to: youtube_data/youtube_comments_caH6q0FA564.csv
2024-10-15 11:42:47,583 - INFO - Starting to scrape comments for video: vjghPIghW-s
2024-10-15 11:42:47,880 - INFO - Scraped 100 comments so far for video vjghPIghW-s...
2024-10-15 11:42:48,121 - INFO - Scraped 158 comments so far for video vjghPIghW-s...
2024-10-15 11:42:48,122 - INFO - Finished scraping comments for video vjghPIghW-s. Total comments: 158
2024-10-15 11:42:48,127 - INFO - Comments for video vjghPIghW-s saved to: youtube_data/youtube_comments_vjghPIghW-s.csv
2024-10-15 11:42:48,129 - INFO - Startin

In [8]:
videos_df

,video_id,title,published_at,channel_title
0,V_dI9KXi2Rs,Steve Kornacki explains Donald Trump pulling e...,2024-10-13T16:16:24Z,NBC News
1,z8CGP0hZ0vo,Jen Psaki breaks down new NBC News poll with H...,2024-10-13T19:01:51Z,MSNBC
2,4QT11_dbjK4,New poll show dead heat race in 2024 president...,2024-10-14T12:35:05Z,CNBC Television
3,C3lIIcaW0nc,US Presidential Elections 2024: Is Harris Losi...,2024-10-14T15:30:06Z,CNN-News18
4,7ABNu6HA9LM,Could Nebraska swing the US elections?,2024-10-13T12:00:20Z,The Times and The Sunday Times
...,...,...,...,...
584,HkwNdEXMqCA,Bernie Sanders Defends Biden’s Fiery Speech on...,2022-11-03T17:59:52Z,NowThis Impact
585,VDFcc1pK1wQ,What happens if a party loses their majority i...,2022-11-04T18:25:34Z,Associated Press
586,ar92fdX7tIc,Lok Sabha Elections 2024: UP में OBC वोटर्स को...,2023-06-09T11:28:44Z,ABP NEWS
587,DbTbMAR8y68,LOOKING FORWARD TO THE AMERICAN 2024 ELECTIONS...,2022-10-07T22:59:38Z,LIVING IN THAILAND ANTIQUES & MILITARIA


In [9]:
comments_df

""
